In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Reading excel file into a DataFrame object.

In [0]:
path="/content/drive/My Drive/RESULTS_Data.xlsx"
dataset=pd.read_excel(path)
dataset.head(10)
print(dataset.columns.values)

['S.NO.' 'ID' 'Subject Code' 'CREDITS' 'MID1 (20M)' 'MID2 (20M)'
 'MID3 (20M)' 'BOM (40M)' 'EST CBT' 'EST PBT' 'ET' 'TOTAL' 'PH'
 'GRAND TOTAL' 'FINAL GRADE']


here some columns names are not in proper way EX:GRAND TOTAL column have the Space between them we replace space with the underscore.Take some empty dictionary for storing the new column values

In [0]:
new_columns={}
for i in dataset.columns.values:
  new_columns[i]=i.replace(' ','_')

print(new_columns)

{'S.NO.': 'S.NO.', 'ID': 'ID', 'Subject Code': 'Subject_Code', 'CREDITS': 'CREDITS', 'MID1 (20M)': 'MID1_(20M)', 'MID2 (20M)': 'MID2_(20M)', 'MID3 (20M)': 'MID3_(20M)', 'BOM (40M)': 'BOM_(40M)', 'EST CBT': 'EST_CBT', 'EST PBT': 'EST_PBT', 'ET': 'ET', 'TOTAL': 'TOTAL', 'PH': 'PH', 'GRAND TOTAL': 'GRAND_TOTAL', 'FINAL GRADE': 'FINAL_GRADE'}


Rename the columns in the Dictionary

In [0]:
dataset.rename(columns=new_columns,inplace=True)
dataset.head(10)

,S.NO.,ID,Subject_Code,CREDITS,MID1_(20M),MID2_(20M),MID3_(20M),BOM_(40M),EST_CBT,EST_PBT,ET,TOTAL,PH,GRAND_TOTAL,FINAL_GRADE
0,1,RKV0001,SUBJ01,4,19,16,11.5,35,10,33,43,78,-,78,A
1,2,RKV0002,SUBJ01,4,17,15,14.5,32,8,23,31,63,-,63,B
2,3,RKV0003,SUBJ01,4,15,17,13,32,7,35,42,74,-,74,B
3,4,RKV0004,SUBJ01,4,15,14,15,30,8,35,43,73,-,73,B
4,5,RKV0005,SUBJ01,4,18,14,13,32,13,31,44,76,-,76,A
5,6,RKV0006,SUBJ01,4,15.5,13,14,29.5,3,30,33,62.5,-,63,B
6,7,RKV0007,SUBJ01,4,16,14,12,30,10,22,32,62,-,62,C
7,8,RKV0008,SUBJ01,4,17,9,7,26,8,22,30,56,-,56,C
8,9,RKV0009,SUBJ01,4,18,15,14.5,33,12,27,39,72,-,72,B
9,10,RKV0010,SUBJ01,4,AB,14,9,23,10,34,44,67,-,67,B


In [0]:
len(dataset)

1107

omit students who are abset to exams,who got remidial and who are tried for malpractices from the datset.

In [0]:
new_dataset=dataset[(dataset["GRAND_TOTAL"]!='AB') & (dataset["GRAND_TOTAL"]!='MP') & (dataset["GRAND_TOTAL"]!='WH')].copy(deep=True)
print(new_dataset.head())
print(len(new_dataset))

   S.NO.       ID Subject_Code  CREDITS  ... TOTAL PH GRAND_TOTAL FINAL_GRADE
0      1  RKV0001       SUBJ01        4  ...    78  -          78           A
1      2  RKV0002       SUBJ01        4  ...    63  -          63           B
2      3  RKV0003       SUBJ01        4  ...    74  -          74           B
3      4  RKV0004       SUBJ01        4  ...    73  -          73           B
4      5  RKV0005       SUBJ01        4  ...    76  -          76           A

[5 rows x 15 columns]
1097


Sorting the DataFrame by Total Marks

In [0]:
new_dataset.sort_values(by='GRAND_TOTAL',ascending=False,inplace=True)
new_dataset.head()

,S.NO.,ID,Subject_Code,CREDITS,MID1_(20M),MID2_(20M),MID3_(20M),BOM_(40M),EST_CBT,EST_PBT,ET,TOTAL,PH,GRAND_TOTAL,FINAL_GRADE
929,930,RKV0930,SUBJ01,4,20,18,19,39,14,42,56,95,-,95,EX
391,392,RKV0392,SUBJ01,4,19,17,19,38,17,39,56,94,-,94,EX
890,891,RKV0891,SUBJ01,4,20,17,19,39,15,38,53,92,-,92,EX
926,927,RKV0927,SUBJ01,4,19,18,19,38,14,40,54,92,-,92,EX
33,34,RKV0034,SUBJ01,4,20,18,18,38,14,40,54,92,-,92,EX


indexes will be shuffeled after sorting applay indexing

In [0]:
new_dataset.reset_index(0,inplace=True)

In [0]:
new_dataset.head(2)

,index,S.NO.,ID,Subject_Code,CREDITS,MID1_(20M),MID2_(20M),MID3_(20M),BOM_(40M),EST_CBT,EST_PBT,ET,TOTAL,PH,GRAND_TOTAL,FINAL_GRADE
0,929,930,RKV0930,SUBJ01,4,20,18,19,39,14,42,56,95,-,95,EX
1,391,392,RKV0392,SUBJ01,4,19,17,19,38,17,39,56,94,-,94,EX


In [0]:
from math import ceil
def cutoff(marks):
  cut={}
  n=len(marks)
  cut["EX"]=min(90,max(80,marks[ceil(5*n/100)-1]))
  cut["A"]=min(80,max(70,marks[ceil(20*n/100)-1]))
  cut["B"]=min(70,max(60,marks[ceil(45*n/100)-1]))
  cut["C"]=min(60,max(50,marks[ceil(15*n/100)-1]))
  cut["D"]=min(50,max(40,marks[ceil(5*n/100)-1]))
  return cut
cut1=cutoff(new_dataset['GRAND_TOTAL'])
print(cut1)

{'EX': 85, 'A': 77, 'B': 69, 'C': 60, 'D': 50}


Now we got the cutoff, we need to read each record of marks data and assign corresponding Grade as per the cutoff.
For which we will use 'apply()' method by passing the cutoff dictionary as additonal parameter.

In [0]:
def AssignGrade(record,cutoff): # Given the Cutoff Dictionary, containing the Grade and its cutoff calculated through 
                                #function getCutOff()
    if record['GRAND_TOTAL'] == 'AB':
        return 'AB'
    elif record['GRAND_TOTAL'] == 'MP':
        return 'MP'
    elif record['GRAND_TOTAL'] == 'WH':
        return 'WH'
    elif record['GRAND_TOTAL'] >= cutoff['EX']:
        return 'EX'
    elif record['GRAND_TOTAL'] >= cutoff['A']:
        return 'A'
    elif record['GRAND_TOTAL'] >= cutoff['B']:
        return 'B'
    elif record['GRAND_TOTAL'] >= cutoff['C']:
        return 'C'
    elif record['GRAND_TOTAL'] >= cutoff['D']:
        return 'D'
    else:
        return 'R'
dataset['New_Grade']=dataset.apply(AssignGrade,axis='columns',args=(cut1,))

print(dataset.head(20))

    S.NO.       ID Subject_Code  CREDITS  ... PH GRAND_TOTAL FINAL_GRADE New_Grade
0       1  RKV0001       SUBJ01        4  ...  -          78           A         A
1       2  RKV0002       SUBJ01        4  ...  -          63           B         C
2       3  RKV0003       SUBJ01        4  ...  -          74           B         B
3       4  RKV0004       SUBJ01        4  ...  -          73           B         B
4       5  RKV0005       SUBJ01        4  ...  -          76           A         B
5       6  RKV0006       SUBJ01        4  ...  -          63           B         C
6       7  RKV0007       SUBJ01        4  ...  -          62           C         C
7       8  RKV0008       SUBJ01        4  ...  -          56           C         D
8       9  RKV0009       SUBJ01        4  ...  -          72           B         B
9      10  RKV0010       SUBJ01        4  ...  -          67           B         C
10     11  RKV0011       SUBJ01        4  ...  -          61           C         C
11  

Writing the resultant DataFrame to the file

In [0]:
dataset.to_excel("/content/drive/My Drive/MYRESULT_Data.xlsx")

to excel files we use the pd.Excelfiles(path)
dataframe.sheets